1. encoder, decoder를 클래스로 만들고 싶었으나 call 함수에서 Input_layer를 설정하는
부분에서 어려움을 겪음

2. 굳이 encoder를 Custom Layer로 만들 필요는 없을 것 같음

2. 일단 이 코드는 안돌아감

# Module Import

In [68]:
# tensorflow 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.layers import Add
from tensorflow.keras import metrics
from tensorflow.keras import losses
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.models import load_model

In [2]:
import numpy as np
import os
import librosa
import librosa.display

In [3]:
test_audio_path = '/home/jsryu1228/crawling/temp/1000.wav'

In [4]:
test_data, sr = librosa.load(test_audio_path, sr=22500, dtype=np.float32)

In [5]:
test_data.shape

(225000,)

In [6]:
# original audio test

from IPython import display as ipd
ipd.Audio(test_data, rate=sr)

# Modeling

In [69]:
class ResidualBlock(layers.Layer):
    def __init__(self, kernel_size, filters, type='encode'):
        super(ResidualBlock, self).__init__()
    
        if type=='encode':
            self.conv1 = layers.Conv1D(filters, kernel_size, 2,padding="same")
            self.conv2 = layers.Conv1D(filters, kernel_size, 1,padding="same")
            self.norm1 = layers.BatchNormalization()
            self.norm2 = layers.BatchNormalization()
        if type=='decode':
            self.conv1 = layers.Conv1DTranspose(filters, kernel_size, 1,padding="same")
            self.conv2 = layers.Conv1DTranspose(filters, kernel_size, 1,padding="same")
            self.norm1 = layers.BatchNormalization()
            self.norm2 = layers.BatchNormalization()
        else:
            return None
            
    def call(self, inputs):
        x = tf.nn.relu(inputs)
        x = self.conv1(x)
        x = self.norm1(x)
        x = layers.LeakyReLU(0.4)(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = layers.LeakyReLU(0.4)(x)

        x = Add()([x, inputs])
        x = tf.nn.relu(x)
        return x      

class Sampling(layers.Layer):

    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = K.random_normal(shape=(K.shape(z_mean)))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
# Labmda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

In [120]:
class Encoder(layers.Layer):
    def __init__(self, input_dim, latent_dim, name="encoder", **kwargs):
        """
        input_dim: sampling_rate * duration
        latent_dim: encoder output dimension
        """
        super(Encoder, self).__init__(name=name, **kwargs)
        self.input_dim = input_dim 
        self.latent_dim = latent_dim
        self.args = {}
        self.args['filters'] = [64, 128, 128, 256]
        self.args['kernel_size'] = [1, 1, 1, 1]
        self.args['strides'] = [2, 2, 2, 2]
        self.args['residual_dim'] = [64, 128, 128, 256]
        self.args['num_layers'] = 4
        self.sampling = Sampling()

    def call(self, inputs): # 여기서 input을 어떻게 설정해야하는지, input_layer를 input의 shape에 맞춰줘도 에러가 발생함
        
        encoder_inputs = layers.Input(shape=(1, self.input_dim))

        x = encoder_inputs

        for idx in range(self.args['num_layers']):
            
            filters = self.args['filters'][idx]
            kernel_size = self.args['kernel_size'][idx]
            stride = self.args['strides'][idx]
            residual_dim = self.args['residual_dim'][idx]
            
            conv_layer = layers.Conv1D(filters, kernel_size, stride, padding="same")
            residual_block = ResidualBlock(residual_dim, kernel_size, type='encode')

            x = conv_layer(x)
            x = residual_block(x)

        x = layers.Flatten()(x)

        z_mean = layers.Dense(self.latent_dim)(x)
        z_log_var = layers.Dense(self.latent_dim)(x)
        z = Sampling()([z_mean, z_log_var])

        return z, z_mean, z_log_var

In [71]:
# encoder 1, 225000

test_data.shape[1:]

# decoder

(1, 225000)

In [122]:
encoder = Encoder(22500*10, 2)

In [73]:
test_data = test_data.reshape(1,22500*10)
test_data = np.expand_dims(test_data, axis=0)

In [74]:
decoder_input = encoder(test_data)[0]

In [82]:
K.int_shape(decoder_input)[1:]

(2,)

In [121]:
# from .utils import ResidualBlock
# from .utils import Sampling

class Decoder(layers.Layer):
    def __init__(self, latent_dim, output_dim, name="decoder", **kwargs):
        """
        latent_dim: encoder output dimension
        output_dim: original input dimension
        """
        super(Decoder, self).__init__(name=name, **kwargs)
        self.latent_dim = latent_dim 
        self.output_dim = output_dim
        self.args = {}
        self.args['filters'] = [512, 256, 128, 64]
        self.args['kernel_size'] = [1, 1, 1, 1]
        self.args['strides'] = [1, 1, 1, 1]
        self.args['residual_dim'] = [512, 256, 128, 64]
        self.args['num_layers'] = 4

    def build(self, inputs):

        decoder_inputs = layers.Input(shape=(self.latent_dim,))

        x = layers.Reshape(target_shape=(1,self.latent_dim))(decoder_inputs)

        for idx in range(self.args['num_layers']):
            
            filters = self.args['filters'][idx]
            kernel_size = self.args['kernel_size'][idx]
            stride = self.args['strides'][idx]
            residual_dim = self.args['residual_dim'][idx]

            residual_block = ResidualBlock(residual_dim, kernel_size, type='decode')            
            convt_layer = layers.Conv1DTranspose(filters, kernel_size, stride, padding="same")

            x = residual_block(x)
            x = convt_layer(x)

        # reconstruction
        decoder_output = layers.Conv1DTranspose(self.output_dim, 1, 1)(x)

        return decoder_output

In [123]:
decoder = Decoder(2, 22500*10)

In [85]:
decoder_output = decoder(decoder_input)

In [86]:
decoder_output.shape

TensorShape([None, 2])

In [ ]:
ipd.Audio(decoder_output.numpy().reshape(-1), rate=sr)

In [134]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


# Data Load

In [88]:
BASE_DIR = '/home/jsryu1228/crawling/temp'

In [126]:
from natsort import natsorted

asmr_list = np.array(natsorted(os.listdir(BASE_DIR)))
train_data = [BASE_DIR + '/%s' % (x) for x in asmr_list]

In [127]:
# 마지막 파일이 10초가 아닐 수 있기 때문에 제거
train_data = train_data[:len(train_data) - 1]

In [128]:
train_data = train_data[1:10]

In [92]:
train_data

['/home/jsryu1228/crawling/temp/1.wav',
 '/home/jsryu1228/crawling/temp/2.wav',
 '/home/jsryu1228/crawling/temp/3.wav',
 '/home/jsryu1228/crawling/temp/4.wav',
 '/home/jsryu1228/crawling/temp/5.wav',
 '/home/jsryu1228/crawling/temp/6.wav',
 '/home/jsryu1228/crawling/temp/7.wav',
 '/home/jsryu1228/crawling/temp/8.wav',
 '/home/jsryu1228/crawling/temp/9.wav']

In [106]:
train_data = tf.data.Dataset.from_tensor_slices(train_data)

In [102]:
def wav_load(asmr):
    y, sampling_rate = librosa.load(asmr, sr=22500, offset=0.0, duration=10)
    y = y.reshape(1,22500*10)
    return y

map_func = lambda file: tf.compat.v1.py_func(wav_load, [file], [tf.float32])

In [119]:
for input_data in train_data.map(map_func).as_numpy_iterator():
    print(input_data[0].shape) # input

(1, 225000)
(1, 225000)
(1, 225000)
(1, 225000)
(1, 225000)
(1, 225000)
(1, 225000)
(1, 225000)
(1, 225000)


In [ ]:
for data in dataset.from_tensor_slices(train_data).map(map_func).batch(8).as_numpy_iterator():
    print(data)

In [93]:
dataset = dataset.from_tensor_slices(train_data).map(map_func).batch(8)

NameError: name 'dataset' is not defined

In [124]:
def wav_load(asmr):
    y, sampling_rate = librosa.load(asmr, sr=22500, offset=0.0, duration=10)
    y = y.reshape(1,22500*10)
    return y

map_func = lambda file: tf.compat.v1.py_func(wav_load, [file], [tf.float32])

In [129]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices(train_data)
    .map(map_func, num_parallel_calls=AUTOTUNE)
    .batch(8)
)

# Callback

In [ ]:
# Model Checkpoint
# asmr 장르가 rain이고 epoch이 50일경우 'vae_rain_50.h5'으로 저장 
check_point_cb = ModelCheckpoint('vae_rain_50.h5', save_best_only=True)

# early stopping
early_stopping_cb = EarlyStopping(patience=5, monitor='total_loss', restore_best_weights=True)

# Learningrate_scheduler


# LearningRateScheduler
lmbda = lambda epoch: epoch ** 0.65
lr_schduler_cb = LearningRateScheduler(lmbda)

# Training

In [135]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.fit(train_dataset,epochs=1, batch_size=32)


#, callbacks=[check_point_cb, early_stopping_cb, lr_schduler_cb])

AttributeError: in user code:

    /home/jsryu1228/anaconda3/envs/asmr/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/jsryu1228/anaconda3/envs/asmr/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/jsryu1228/anaconda3/envs/asmr/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/jsryu1228/anaconda3/envs/asmr/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/jsryu1228/anaconda3/envs/asmr/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)

# Sampling

In [ ]:
def generate_asmr(model, test_data):
    
    # original audio
    librosa.display.waveplot(test_data, sr=22050)
    
    z, mean, log_var = model.encode(test_data)
    reconstructed_signal = model.decode(z)
    
    # reconstructed audio
    reconstructed_signal = reconstructed_signal.numpy().reshape(-1)
    librosa.display.waveplot(reconstructed_signal, sr=22050)

In [ ]:
lib